<a href="https://colab.research.google.com/github/DarioRugg/KickLearning/blob/main/collaborators_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from os.path import join
import pandas as pd
import numpy as np
data_path = join('drive', 'MyDrive', 'Project', 'Data', 'complete_scraped_df.csv')
df = pd.read_csv(data_path)

In [ ]:
new_col = [eval(collabs) if type(collabs) == str else [] for collabs in df['collab_names']]
df['collab_names'] = new_col
all_collabs = {col for collabs in df['collab_names'] for col in collabs}


In [ ]:
# assigning to each collaborator a list of tuples with information from projects
# they have worked to, including the creation time of the project
collabs_projects = {col:[] for col in all_collabs}
for i, project in df.iterrows():
  for col in project['collab_names']:
    collabs_projects[col].append((project['usd_pledged'], project['state'], project['creator_id'], project['created_at']))

In [ ]:
# for each project an average score for all professional collaborators that have worked 
# to the project
for i, project in df.iterrows():
  pro_collabs = []
  for col in project['collab_names']:
    older_projects = [proj for proj in collabs_projects[col] if proj[3] <= project['created_at']]
    pledge_tot = []
    state_tot = []
    creators = set()
    for proj in older_projects:
      if proj[1] in {'successful', 'failed'}:
        pledge_tot.append(proj[0])
        state_tot.append(proj[1] == 'successful')
        creators.add(proj[2])
    n_creators = len(creators)
    # a professional collaborator has worked with at least 2 different creators 
    if n_creators >= 2:
      pro_collabs.append((np.mean(pledge_tot)*np.mean(state_tot)))
  df.loc[i, 'collabs_avg_score'] = np.mean(pro_collabs)



/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
df.to_csv(join('drive', 'MyDrive', 'Project', 'Data','collabs_df.csv'), index=False)